In [78]:
import pandas as pd
import numpy as np
import requests
import json
from pandas import json_normalize
from requests.auth import HTTPBasicAuth

from dotenv.main import load_dotenv
import os



pd.options.display.max_seq_items = 20000
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)

In [4]:
#Load the Api Key

from dotenv.main import load_dotenv
import os
load_dotenv()
API_KEY=os.environ["api_key"]

### 0.1 Notes and Thoughts

Hard Filters that are needed for the Dataset are:
- B2C
- Europe 
- (Later industry, first iteration without industry specificat)

All Available fields:
id
name
type
deleted
path
tagline
about
url,website_url,twitter_url,facebook_url,linkedin_url,google_url,crunchbase_url,angellist_url
playmarket_app_id,appstore_app_id
images
employees,employees_latest
industries,sub_industries,corporate_industries,service_industries
technologies
income_streams
growth_stage
traffic_summary
hq_locations
client_focus
revenues
tags
ownerships
delivery_method
launch_year,launch_month
has_promising_founder,has_strong_founder,has_super_founder
total_funding,total_funding_source,last_funding,last_funding_source
company_status,last_updated,last_updated_utc,created_utc
facebook_likes_chart,twitter_tweets_chart,twitter_followers_chart,twitter_favorites_chart
employees_chart
similarweb_3_months_growth_unique,similarweb_3_months_growth_percentile,similarweb_3_months_growth_relative,similarweb_3_months_growth_delta,similarweb_6_months_growth_unique,similarweb_6_months_growth_percentile,similarweb_6_months_growth_relative,similarweb_6_months_growth_delta,similarweb_12_months_growth_unique,similarweb_12_months_growth_percentile,similarweb_12_months_growth_relative,similarweb_12_months_growth_delta

app_3_months_growth_unique,app_3_months_growth_percentile,app_3_months_growth_relative,app_6_months_growth_unique,app_6_months_growth_percentile,app_6_months_growth_relative,app_12_months_growth_unique,app_12_months_growth_percentile,app_12_months_growth_relative,employee_3_months_growth_unique

employee_3_months_growth_percentile,employee_3_months_growth_relative,employee_3_months_growth_delta,employee_6_months_growth_unique,employee_6_months_growth_percentile,employee_6_months_growth_relative,employee_6_months_growth_delta,employee_12_months_growth_unique,employee_12_months_growth_percentile,employee_12_months_growth_relative,employee_12_months_growth_delta
kpi_summary
team
investors
fundings
traffic
similarweb_chart
job_openings
exits
trading_multiple
app_downloads_ios_chart,app_downloads_android_chart,app_downloads_ios_incremental_chart,app_downloads_android_incremental_chart
tech_stack_predictleads
sustainable_development_goals
core_side_value
data_type
pic_number
patents_count


### 0.2 Getting Available Filters for the Post Requests

In [289]:

api_url="https://api.dealroom.co/api/v1/companies/filters"
auth=HTTPBasicAuth(API_KEY, '')

headers = {'Content-Type': 'application/json'}
data={"fields":"launch_year",
              "limit": 1,
              "offset": 0
             }
response = requests.post(api_url,data=json.dumps(data),auth=auth,headers=headers)
data=response.json()
data


{'items': [{'key': 'launch_year',
   'items': [{'name': 2015, 'count': 84335, 'active': False},
    {'name': 2016, 'count': 79793, 'active': False},
    {'name': 2014, 'count': 77962, 'active': False},
    {'name': 2017, 'count': 76836, 'active': False},
    {'name': 2018, 'count': 72570, 'active': False},
    {'name': 2013, 'count': 72278, 'active': False},
    {'name': 2019, 'count': 69834, 'active': False},
    {'name': 2012, 'count': 68595, 'active': False},
    {'name': 2020, 'count': 62083, 'active': False},
    {'name': 2011, 'count': 58551, 'active': False},
    {'name': 2010, 'count': 55704, 'active': False},
    {'name': 2009, 'count': 46991, 'active': False},
    {'name': 2008, 'count': 40860, 'active': False},
    {'name': 2007, 'count': 37910, 'active': False},
    {'name': 2021, 'count': 37737, 'active': False},
    {'name': 2006, 'count': 33847, 'active': False},
    {'name': 2005, 'count': 30030, 'active': False},
    {'name': 2004, 'count': 27234, 'active': False},
   

# 1.1 Building the Core DF

1. Variables for the DF:

1.1: Independent Variables
- Team
- patents (patents_count)
- Round (fundings)
- Investors
- employees (latest, chart)
- Num. Investors (investors)
- (Industry Category) (industries)
- Description Length (about)
- Has Domain (website_url)
- Has Facebook (facebook_url)
- Has Twitter (twitter_url)
- Has LinkedIn (linkedin_url)
- Geographical Region (country, city/-code) (hq_locations)
- Total Funding (total_funding) (in mio)

1.2 Dependent Variables:
- Stage


Goal:
Predict possibility of follow on funding within the time frame of 3 years
Alternative: Predict possibility of funding, fail, acquisition, public

Rows that are dropped with empty values in following columns:
1. Web Url 
    avoid ghost companies
2. ownerships 
    bootstraped companies will not have funding information, thus not a missing value
    also essential for dependend variable
3. company status
    little info missing
4. launch year
    essential to project data to past
5. team items
    Intuitively most essential for predictive power

### Importing the Json file through according API Call:

In [290]:
fields="id,name,about,tagline,investors,industries,website_url,facebook_url,twitter_url,linkedin_url,hq_locations,total_funding,company_status,ownerships,team,fundings,patents_count,employees_latest,employees_chart,launch_year,dealroom_signal"
api_url="https://api.dealroom.co/api/v1/companies/bulk"
auth=HTTPBasicAuth(API_KEY, '')
headers = {'Content-Type': 'application/json'}

json_list=[]
next_page_id = ''
k=0

while True:
    data = {
        'form_data': {'must':{'hq_locations': ['Europe'], 'client_focus':['business']}},
        'fields': fields,
        'next_page_id': next_page_id,
        'limit': 100,
        }

    headers = {'Content-Type': 'application/json'}

    response = requests.post(api_url,data=json.dumps(data),auth=auth,headers=headers)

    res=json.loads(response.text)
    json_list.append(res)
    k+=1
    if (k/10).is_integer():
        print(k,end="\r")
    next_page_id = res['next_page_id']

    if next_page_id == None:
        break

In [291]:
with open("first_json.json", "w") as fp:
        json.dump(json_list, fp)

In [25]:
f=open("/Users/klaus/Documents/Jarvis/Second Try/Data/first_json.json")
json_list=json.load(f)

### 1.2 Converting Json to Pandas DF

In [292]:
df_list=[]
for i in range(len(json_list)):
    df_1=pd.json_normalize(json_list[i]["items"],sep="_").drop(columns=["hq_locations","industries"])
    df_1[["country_name","city_name"]]=pd.json_normalize(json_list[0]["items"],"hq_locations")[["country.name","city.name"]]
    df_1["industry_name"]=pd.json_normalize(json_list[0]["items"],"industries")["name"]
    df_list.append(df_1)

In [293]:
df=pd.concat(df_list)

In [294]:
df.shape

(222192, 25)

## 1.3 DataCleaning

In [295]:
x=df.isna().sum()
y=df.isna().sum()/len(df)*100
missing_values=pd.concat([x, y], axis=1)
missing_values.columns=["Missing Values","In percent"]
print(missing_values)


                  Missing Values  In percent
id                             0    0.000000
name                           0    0.000000
about                      57595   25.921275
tagline                    63302   28.489775
website_url                  616    0.277238
facebook_url              175755   79.100508
twitter_url               112862   50.794808
linkedin_url               50588   22.767696
total_funding                  0    0.000000
company_status                84    0.037805
ownerships                     0    0.000000
patents_count                  0    0.000000
employees_latest           59830   26.927162
employees_chart                0    0.000000
launch_year                34707   15.620274
dealroom_signal           177722   79.985778
investors_items                0    0.000000
investors_total                0    0.000000
team_items                     0    0.000000
team_total                     0    0.000000
fundings_items                 0    0.000000
fundings_t

In [296]:
df.dropna(subset=["website_url"],inplace=True) #companies without website will be deleted to avoid ghost companies
df.dropna(subset=["company_status"],inplace=True)
df.dropna(subset=["launch_year"],inplace=True)#companies without status (only 85 so can be dismissed)

### Number of Empty lists as entries

In [299]:
for i,row in missing_values.iterrows():
    try:
        print(i,df[i].apply(lambda x:1 if len(x) == 0 else 0).sum()/len(df)*100,"%")
    except:
        None



name 0.0 %
website_url 0.0 %
company_status 0.0 %
ownerships 52.72182863253576 %
employees_chart 20.076995054137146 %
investors_items 72.37588557679454 %
team_items 56.14971260526668 %
fundings_items 72.35931025264001 %
country_name 0.0 %


In [313]:
df_1=df[df["ownerships"].apply(lambda x:False if len(x) == 0 else True)]


In [314]:
for i,row in missing_values.iterrows():
    try:
        print(i,df_1[i].apply(lambda x:1 if len(x) == 0 else 0).sum()/len(df_1)*100,"%")
    except:
        None  

print(df_1.shape)

name 0.0 %
website_url 0.0 %
company_status 0.0 %
ownerships 0.0 %
employees_chart 11.918979439506005 %
investors_items 59.88554884530999 %
team_items 18.245459274841103 %
fundings_items 51.88414647938296 %
country_name 0.0 %
(88422, 25)


### Further clean Dataset by removing all Empty lists in the team column

In [315]:
df_1=df_1[df_1["team_items"].apply(lambda x:False if len(x) == 0 else True)]

In [317]:
for i,row in missing_values.iterrows():
    try:
        print(i,df_1[i].apply(lambda x:1 if len(x) == 0 else 0).sum()/len(df_1)*100,"%")
    except:
        None

name 0.0 %
website_url 0.0 %
company_status 0.0 %
ownerships 0.0 %
employees_chart 8.069000816168435 %
investors_items 62.691419164741525 %
team_items 0.0 %
fundings_items 56.99899016447869 %
country_name 0.0 %


### We observe that many of the empty lists for fundings and investors are for bootstrapped startups. It makes sense to distinguish between those where data is missing and those where it simply makes no sense to have data

In [372]:
df_invest=df_1.loc[df_1["investors_items"].apply(lambda x:True if len(x) == 0 else False)]
df_boots=df_invest.loc[df_invest["ownerships"].apply(lambda x:True if ((x[0]=="bootstrapped" and len(x)<=1) or (x==["not known","bootstrapped"]) or (x[0]=="not known")) else False)]

df_boots["investors_items"].values[:]="nan"

df_2=pd.concat([df_1,df_boots])

df_2.drop_duplicates(subset=["id"],keep="last", inplace=True)

In [373]:
df_fundings=df_2.loc[df_2["fundings_items"].apply(lambda x:True if len(x) == 0 else False)]
df_fundinv=df_fundings.loc[df_fundings["investors_items"].apply(lambda x:True if x == "nan" else False)]
df_shit=df_fundinv.loc[df_fundinv["ownerships"].apply(lambda x:True if ((x[0]=="bootstrapped" and len(x)<=1) or (x==["not known","bootstrapped"]) or (x[0]=="not known")) else False)]

df_shit["fundings_items"].values[:]="nan"

df_3=pd.concat([df_2,df_shit])

df_3.drop_duplicates(subset=["id"],keep="last", inplace=True)

In [361]:
x=df_3.isna().sum()
y=df_3.isna().sum()/len(df_3)*100
missing_values=pd.concat([x, y], axis=1)
missing_values.columns=["Missing Values","In percent"]
print(missing_values)

                  Missing Values  In percent
id                             0    0.000000
name                           0    0.000000
about                       5020    6.944348
tagline                    11887   16.443719
website_url                    0    0.000000
facebook_url               47439   65.624092
twitter_url                23527   32.545754
linkedin_url                4784    6.617881
total_funding                  0    0.000000
company_status                 0    0.000000
ownerships                     0    0.000000
patents_count                  0    0.000000
employees_latest            5833    8.069001
employees_chart                0    0.000000
launch_year                    0    0.000000
dealroom_signal            50183   69.419967
investors_items            41383   57.246607
investors_total                0    0.000000
team_items                     0    0.000000
team_total                     0    0.000000
fundings_items                 0    0.000000
fundings_t

In [375]:
for i,row in missing_values.iterrows():
    try:
        print(i,df_3[i].apply(lambda x:1 if len(x)==0 else 0).sum()/len(df_3)*100,"%")
    except:
        print(i,"0.0%")
        

id 0.0%
name 0.0 %
about 0.0%
tagline 0.0%
website_url 0.0 %
facebook_url 0.0%
twitter_url 0.0%
linkedin_url 0.0%
total_funding 0.0%
company_status 0.0 %
ownerships 0.0 %
patents_count 0.0%
employees_latest 0.0%
employees_chart 8.069000816168435 %
launch_year 0.0%
dealroom_signal 0.0%
investors_items 5.444811797092227 %
investors_total 0.0%
team_items 0.0 %
team_total 0.0%
fundings_items 1.5866867711546706 %
fundings_total 0.0%
country_name 0.0 %
city_name 0.0%
industry_name 0.0%


Data Seems quite clean to me. No more Data Cleaning Needed 

### 1.3 Unravel nested Data to generate Present Dataset

For the following we want to extract the nested data and retreive the information we need
First we extract investors types

In [451]:
def level1_extractor_list(x,field):
    try:
        a=list(pd.unique(pd.json_normalize(x)[field]))
    except:
        a=[]
    return a

def level1_extractor_max(x,field):
    try:
        a=max(list(pd.json_normalize(x)[field]))
    except:
        a=[]
    return a

def level2_extractor_list(x,field1,field2):
    try:
        first=pd.json_normalize(x)[field1]
        mp=map(lambda p: level1_extractor_list(p, field2), first)
        a=list(pd.unique([c for b in list(mp) if len(b)!=0 for c in b]))
    except:
        a=[]
    return a

def level2_extractor_max(x,field1,field2):
    try:
        first=pd.json_normalize(x)[field1]
        mp=map(lambda p: level1_extractor_list(p, field2), first)
        a=max([x for x in list(mp) if x!=None])
    except:
        a=None
    return a

def level2_extractor_min(x,field1,field2):
    try:
        first=pd.json_normalize(x)[field1]
        mp=map(lambda p: level1_extractor_list(p, field2), first)
        a=min([x for x in list(mp) if x!=None])
    except:
        a=None
    return a

def level2_extractor_avg(x,field1,field2):
    try:
        first=pd.json_normalize(x)[field1]
        mp=map(lambda p: level1_extractor_list(p, field2), first)
        a=np.mean([x for x in list(mp) if x!=None])
    except:
        a=None
    return a

Export Feature of Universities at which team members studies

In [452]:
a=map(lambda p: level2_extractor_list(p, "universities.items","university.name"), df_3["team_items"])
b=list(a)

df_3["team_universities"]=b

Export Features of Investor Types

In [421]:
a=map(lambda p: level1_extractor_list(p, "type"), df_3["investors_items"])
b=list(a)
df_3["investors_types"]=b


# Save of File

In [166]:
#Save Dataframe
df_3.to_pickle("df_25.08")

In [5]:
df_3=pd.read_pickle("/Users/klaus/Documents/Jarvis/Second Try/Data/df_25.08")

# Continue

Export Features of Valuation

In [170]:
a=map(lambda p: level1_extractor_list(p, "valuation_generated_max"), df_3["fundings_items"])
b=list(a)
df_3["list_valuation"]=b
df_3["last_valuation"]=list(map(lambda p: p[0] if p!=None else  None,df_3["list_valuation"]))
df_3.drop(columns="list_valuation",inplace=True)

Get Number of Female and Number of Male Founder

In [36]:
a=map(lambda p: level1_extractor_list(p, "gender"), df_3["team_items"])
b=list(a)
df_3["team_gender"]=b
df_3["team_male"]=list(map(lambda p: p.count("male"),df_3["team_gender"]))
df_3["team_female"]=list(map(lambda p: p.count("female"),df_3["team_gender"]))
df_3.drop(columns="team_gender",inplace=True)

Turning url's to the binary (0-has no url, 1-has url)

In [204]:
df_3["website_url"]=df_3["website_url"].apply(lambda x:0 if x==None else 1)
df_3["linkedin_url"]=df_3["linkedin_url"].apply(lambda x:0 if x==None else 1)
df_3["facebook_url"]=df_3["facebook_url"].apply(lambda x:0 if x==None else 1)
df_3["twitter_url"]=df_3["twitter_url"].apply(lambda x:0 if x==None else 1)

### Spliting the Data into on dataframe from 3 years ago and today

First lets try to exclude all companies that have been founded in the between time

- Variables that need to be put in past
0. Number Employees (maybe include growth over the past two to four entries)
1. Team universities
2. Gender of Team
3. Stage of company
4. max valuation
5. investors total
6. fundings total
7. total funding
8. company status

In [376]:
def level1_extractor_list_past(x,year_field,year,field):
    try:
        a=pd.json_normalize(x)
        a=list(a[a[year_field]<=year][field])
    except:
        a=None
    return a

def level2_extractor_list_past1(x,year_field,year,field1,field2):
    try:
        first=pd.json_normalize(x)
        first=first[first[year_field]<=year][field1]
        mp=map(lambda p: level1_extractor_list(p, field2), first)
        a=list(pd.unique([x for x in list(mp) if x!=None]))
    except:
        a=None
    return a

def level2_extractor_list_past2(x,year_field,year,field1,field2):
    try:
        first=pd.json_normalize(x)[field1]
        mp=map(lambda p: level1_extractor_list_past(p,year_field,year, field2), first)
        a=list(pd.unique([x for x in list(mp) if x!=None]))
    except:
        a=None
    return a

Get DF copy with companies launched before 2020

In [378]:
df_past=df_3[df_3["launch_year"]<=2020].copy()

(70211, 25)

0. Number of Employees

In [379]:
a=map(lambda p: level1_extractor_list_past(p,"date","2020-01-01","value"), df_past["employees_chart"])
b=list(a)
df_past["employees_latest"]=b


1. Team Universities

I will include all universities at this point despite the position the person has. I believe that even a software developer that graduated from a good school has a positive impact on the company. This is also an indicator that the startup is able to attract top talent

In [464]:
a=map(lambda p: level2_extractor_list(p, "universities.items","university.name"), df_past["team_items"])
b=list(a)

df_past["team_universities"]=b

2. Gender of Team

Only include the gender of the top Exec's

In [462]:
a=map(lambda p: level2_extractor_list(p, "titles","name"), df_past["team_items"])
b=list(a)

df_past["positions"]=b

In [476]:
a=pd.unique(df_past.positions.sum())

In [487]:
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/usr/local/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [519]:
titles_wanted=["ceo","cfo","cto","chief","founder","officer","leader","owner"]
titles_filter=[x for x in a if process.extractOne(x.lower(), titles_wanted, scorer=fuzz.token_sort_ratio)[-1]>=60]


In [582]:
def level1_extractor_list_gender(x,titles_wanted):
    try:
        a=pd.json_normalize(x)
        a["title_name"]=list(map(lambda p: level1_extractor_list(p,"name"),a["titles"]))
        try:
            a=a[a["title_name"].apply(lambda x:True if (len(x)!=0 and x[0] in titles_filter) else False)]
        except:
            None
        a=list(a["gender"])        
    except:
        a=None
    return a

In [592]:
a=map(lambda p: level1_extractor_list_gender(p, titles_filter), df_past["team_items"])
b=list(a)

df_past["gender"]=b

In [595]:
df_past["team_male"]=list(map(lambda p: p.count("male"),df_past["gender"]))
df_past["team_female"]=list(map(lambda p: p.count("female"),df_past["gender"]))
df_past.drop(columns="gender",inplace=True)

In [597]:
#Save Dataframe
df_past.to_pickle("df_past_29.08")

Generate List for LinkedIn Scraper to get Universities of Founders

In [57]:
df_missing_univ=df_3[df_3["team_universities"].apply(lambda x:True if len(x)==0 else False)].copy()
df_wuniv=df_3[df_3["team_universities"].apply(lambda x:False if len(x)==0 else True)].copy()

In [54]:
df_missing_univ.shape

(27035, 28)

In [130]:
a=map(lambda p: level1_extractor_list(p, "linkedin_url"), df_missing_univ["team_items"])
b=list(a)
df_missing_univ["linkedin_url_founder"]=b
df_missing_univ["linkedin_url_founder"]=list(map(lambda p: p[0],df_missing_univ["linkedin_url_founder"]))

In [131]:
df_missing_univ.head(n=20)

,id,name,about,tagline,website_url,facebook_url,twitter_url,linkedin_url,total_funding,company_status,ownerships,patents_count,employees_latest,employees_chart,investors_items,investors_total,team_items,team_total,fundings_items,fundings_total,country_name,city_name,industry_name,investors_types,team_universities,linkedin_url_founder
82,2526214,Phoenix Eleven,We empower entrepreneurs giving them the tools...,Phoenix Eleven Capital - The Human Touch,1,0,0,1,0.00,operational,[bootstrapped],0,2.0,"[{'date': '2020-12-01', 'value': 3}, {'date': ...","[{'id': 3214034, 'name': 'Flori Ventures', 'pa...",1,"[{'id': 2969484, 'name': 'Nick Allison', 'path...",2,[],0,Germany,Ravensburg,semiconductors,[fund],[],https://www.linkedin.com/in/pecapital/
84,2441129,Pickar,We believe that finding yourself in a career y...,Developed a psychometric analysis alongside ex...,1,0,0,1,0.00,operational,[Venture Capital],0,1.0,"[{'date': '2021-05-11', 'value': 2}, {'date': ...","[{'id': 935917, 'name': 'Georgia Institute of ...",1,"[{'id': 3831057, 'name': 'Julian Willner', 'pa...",2,"[{'id': 427762, 'year': 2019, 'month': 6, 'amo...",2,Switzerland,Eysins,media,[university],[],https://www.linkedin.com/in/julian-willner/
89,155526,QRcao,None,Co-founder and product engineer @ Appvetica | ...,1,1,1,1,0.01,closed,[Venture Capital],0,NaN,[],[],0,"[{'id': 1044949, 'name': 'Peter Tuszynski', 'p...",1,"[{'id': 200613, 'year': 2011, 'month': 8, 'amo...",1,United Kingdom,London,robotics,nan,[],https://www.linkedin.com/in/petertuszynski/
97,149265,Tech'4'Team,Arenametrix is a SaaS B2B start-up created in ...,None,1,0,1,1,0.60,operational,"[Angel, Venture Capital, accelerator]",0,26.0,"[{'date': '2018-07-16', 'value': 19}, {'date':...","[{'id': 885468, 'name': 'Paris&Co', 'path': 'i...",2,"[{'id': 349656, 'name': 'Vitoz Kévin', 'path':...",6,"[{'id': 222831, 'year': 2016, 'month': 4, 'amo...",2,United Kingdom,NaN,NaN,"[fund, investor]",[],None
11,118206,Newswired,None,Digital publishing platform that enables publi...,1,1,1,1,0.02,closed,[Venture Capital],0,NaN,[],[],0,"[{'id': 1135128, 'name': 'Andrey Aleshkov', 'p...",2,"[{'id': 202314, 'year': 2013, 'month': 12, 'am...",1,United Kingdom,Great and Little Hampden,transportation,nan,[],https://www.linkedin.com/in/andreyaleshkov/
97,97110,RentBid,Mechanics Mechanics Pick Your yesMechanics Vie...,None,1,0,1,1,0.00,closed,[Venture Capital],0,NaN,[],"[{'id': 908352, 'name': 'Rev1 Ventures', 'path...",1,"[{'id': 255915, 'name': 'Alexander Willcox', '...",2,"[{'id': 288441, 'year': 2015, 'month': 5, 'amo...",1,United Kingdom,NaN,NaN,[fund],[],None
99,98045,Overture Services,Home | Yahoo Advertising Home Mail Search New...,Online advertising services via its network of...,1,1,1,1,0.00,acquired,"[Subsidiary, Angel, Venture Capital]",0,NaN,[],"[{'id': 20810, 'name': 'Upfront Ventures', 'pa...",4,"[{'id': 258014, 'name': 'Laurence Aderemi', 'p...",28,"[{'id': 226763, 'year': 2003, 'month': 7, 'amo...",3,United Kingdom,West Suffolk,NaN,"[fund, corporate, investor, fund]",[],None
1,99046,Coupoplaces,None,A platform where the customers of daily deal c...,1,0,1,1,0.01,closed,[Venture Capital],0,NaN,[],[],0,"[{'id': 1104205, 'name': 'Alex Petrov', 'path'...",3,"[{'id': 15405, 'year': 2011, 'month': 6, 'amou...",1,Russia,Dimitrovgrad,jobs recruitment,nan,[],None
4,99372,Socialance,Socialance is a professional network that conn...,Professional network for outsourcing,1,1,1,1,0.04,operational,[Venture Capital],0,NaN,[],"[{'id': 3654, 'name': 'Start-Up Chile', 'path'...",1,"[{'id': 1148514, 'name': 'Guillermo Vigil', 'p...",2,"[{'id': 15278, 'year': 2012, 'month': 8, 'amou...",1,Germany,Münster,transportation,[fund],[],None
33,223114,Quaturo,BlueGlass | SEO Content Marketing Agency in ...,BlueGlass | SEO & Content Marketing Agency in ...,1,1,1,1,0.00,acquired,[Subsidiary],0,NaN,[],"[{'id': 7794, 'name': 'BlueGlass Interactive',...",1,"[{'id': 39622, 'name': 'Kevin Gibbons', 'path'...",1,"[{'id': 223356, 'year': 2012, 'month': 11, 'am.

In [132]:
df_missing_univ["linkedin_url_founder"].count()

12666